# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import pickle
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Reading Data From CSV File

In [2]:
data = pd.read_csv('flipkart.csv', low_memory=False, encoding='utf-8')

# Creating a NEW dataframe from acutal dataframe by selecting specfic columns

In [3]:
df = pd.DataFrame(data[['description','brand']])
df

,description,brand
0,Key Features of Alisha Solid Women's Cycling S...,Alisha
1,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor
2,Key Features of AW Bellies Sandals Wedges Heel...,AW
3,Key Features of Alisha Solid Women's Cycling S...,Alisha
4,Specifications of Sicons All Purpose Arnica Do...,Sicons
...,...,...
19995,Buy WallDesign Small Vinyl Sticker for Rs.730 ...,WallDesign
19996,Buy Wallmantra Large Vinyl Stickers Sticker fo...,Wallmantra
19997,Buy Elite Collection Medium Acrylic Sticker fo...,Elite Collection
19998,Buy Elite Collection Medium Acrylic Sticker fo...,Elite Collection


# Cleaning Messy Column Names

In [4]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
df.columns

Index(['description', 'brand'], dtype='object')

# Removing Special Characters from Dataframe

In [5]:
# Replacing dataFrame in another Variable.
df_spsl = df

# Declaration all Sorts of Special Characters in the form of List.
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–", "//", "%*", ":/", ".;", "Ø", "§"]

# Checking
for c in spec_chars:
    c1 = "\\" + c
    # Replacing With SPACE"".
    df_spsl=df_spsl.replace(c1,"", regex=True)

# Replacing DataFrame with no Special characters.
df1 = df_spsl
display(df1)

,description,brand
0,Key Features of Alisha Solid Womens Cycling Sh...,Alisha
1,FabHomeDecor Fabric Double Sofa Bed Finish Col...,FabHomeDecor
2,Key Features of AW Bellies Sandals Wedges Heel...,AW
3,Key Features of Alisha Solid Womens Cycling Sh...,Alisha
4,Specifications of Sicons All Purpose Arnica Do...,Sicons
...,...,...
19995,Buy WallDesign Small Vinyl Sticker for Rs730 o...,WallDesign
19996,Buy Wallmantra Large Vinyl Stickers Sticker fo...,Wallmantra
19997,Buy Elite Collection Medium Acrylic Sticker fo...,Elite Collection
19998,Buy Elite Collection Medium Acrylic Sticker fo...,Elite Collection


# Creating a custom class to Impute Null values using TransformerMixin

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

df = DataFrameImputer().fit_transform(df1)
display(df)    

,description,brand
0,Key Features of Alisha Solid Womens Cycling Sh...,Alisha
1,FabHomeDecor Fabric Double Sofa Bed Finish Col...,FabHomeDecor
2,Key Features of AW Bellies Sandals Wedges Heel...,AW
3,Key Features of Alisha Solid Womens Cycling Sh...,Alisha
4,Specifications of Sicons All Purpose Arnica Do...,Sicons
...,...,...
19995,Buy WallDesign Small Vinyl Sticker for Rs730 o...,WallDesign
19996,Buy Wallmantra Large Vinyl Stickers Sticker fo...,Wallmantra
19997,Buy Elite Collection Medium Acrylic Sticker fo...,Elite Collection
19998,Buy Elite Collection Medium Acrylic Sticker fo...,Elite Collection


# Sepereating Fetures as X and y

In [7]:
X, y = df.description, df.brand

# Text Preprocessing

In [8]:
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)
    

In [9]:
display(documents)

['key feature of alisha solid woman cycling short cotton lycra navy red navyspecifications of alisha solid woman cycling short short detail number of content in sale package pack of 3 fabric cotton lycra type cycling short general detail pattern solid ideal for woman fabric care gentle machine wash in lukewarm water do not bleach additional detail style code altht3p21 in the box 3 short',
 'fabhomedecor fabric double sofa bed finish color leatherette black mechanism type pull out price r 22646 fine deep seating experience save space with the all new click clack sofa bed easy to fold and vice versa with simple click clack mechanism chrome leg with mango wood frame for long term durability double cushioned sofa bed to provide you with extra softness to make fine seating experience double bed that can easily sleep twospecifications of fabhomedecor fabric double sofa bed finish color leatherette black mechanism type pull out installation demo installation demo detail installation and demo 

# Bag of Words (BoW)

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from yellowbrick.text import FreqDistVisualizer
vectorizer = CountVectorizer(max_features=10, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))

# X = vectorizer.fit_transform(documents).toarray()

docs       = vectorizer.fit_transform(documents)
features   = vectorizer.get_feature_names()

visualizer = FreqDistVisualizer(features=features, orient='v')
visualizer.fit(docs)
visualizer.show()


ModuleNotFoundError: No module named 'yellowbrick'

# Finding TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()
X

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))